# CNNs


## Simple Convolutional Layer


- 一些细节：
  - python中的sum
    - sum(): 计算所有元素相加
    - sum(axis=0): 计算每一列之和，返回一行
    - sum(axis=1): 计算每一行之和，返回一列
  - torch中的sum(input, dim, keepdim=False): dim是the dimension or dimensions to reduce，如果是1，那么就消除列，变成一列（x，1），也可以理解为在列的角度进行sum，对每一行，所有列相加。与python的sum是一样。
  - torch.argmax()也是这样。
  - CNN中卷积层实际上是互相关计算，卷积的矩阵是互相关矩阵的上下、左右反转，但这无所谓，因为不管是卷积还是互相关，里面的参数都是学出来的，所以用哪种方式学出来的矩阵和输入的乘积结果都是不变的。
    

In [1]:
import torch
from torch import nn as nn


In [2]:
# The calculation of convolution
def conv(X,Kernel):
    h,w = Kernel.shape
    Y = torch.zeros(X.shape[0] - h + 1, X.shape[1] - w + 1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+h,j:j+w] * Kernel).sum()
    return Y

X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = torch.tensor([[0, 1], [2, 3]])
conv(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [3]:
# The layer of convolution
class conv_layer(nn.Module):
    def __init__(self, kernel_size):
        super(conv_layer,self).__init__()
        self.kernel = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1))
        
    def forward(self, X):
        Y = conv(X, self.kernel) + self.bias
        return Y

X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = [2,2]  

net = conv_layer(K)
print(net)
print(net(X))



conv_layer()
tensor([[-3.0843, -1.9405],
        [ 0.3472,  1.4910]], grad_fn=<AddBackward0>)


In [4]:
# An example to perform convolution
X = torch.ones(6, 8)
X[:, 2:6] = 0
print('X:',X)

K = torch.tensor([[1, -1]])
Y = conv(X, K)
print('Y',Y)

X: tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])
Y tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])


In [12]:
# Learning process for kernel parameters
net = conv_layer(kernel_size=(1,2))

step = 20
lr = 0.01
for i in range(step):
    Y_hat = net(X)
    loss = ((Y_hat-Y)**2).sum()
    loss.backward()
    net.kernel.data -= lr*net.kernel.grad  # use -= to keep the change
    net.bias.data -= lr*net.bias.grad
    
    #net.kernel.grad.fill_(0)
    #net.bias.grad.fill_(0)
    net.kernel.grad.zero_()
    net.bias.grad.zero_()
    if (i+1) % 5 ==0:
        print('step and loss',i+1,loss.item())
    
    
print('kernel:', net.kernel.data)
print('bias:', net.bias.data)
    

step and loss 5 18.151744842529297
step and loss 10 2.075840473175049
step and loss 15 0.24572691321372986
step and loss 20 0.03135181963443756
kernel: tensor([[ 1.0017, -0.9571]])
bias: tensor([-0.0250])
